In [1]:
# imports
import pandas as pd

## Machine Learning

### Preparation of the data for model building

#### Data loading

In [8]:
data = pd.read_csv("https://raw.githubusercontent.com/EY-Tech-Consulting-Denmark/Graphathon-ATP/main/Data/clean_data/data.csv")
data.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ClmProcedureCode_5,PotentialFraud,Age,IsDead,DaysAdmitted,DaysClaimLasted,ClaimEndAfterDischarged,TotalDiagnosis,TotalProcedures,TotalPhysicians
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,NaN,1,66.0,0,7,7,0,9,0,1
1,BENE17521,1913-12-01,NaN,2,1,0,39,230,12,12,...,NaN,1,95.0,0,13,13,0,9,2,3
2,BENE21718,1922-10-01,NaN,1,1,0,39,600,12,12,...,NaN,1,87.0,0,19,19,0,9,1,2
3,BENE22934,1930-07-01,NaN,2,1,0,39,280,12,12,...,NaN,1,79.0,0,5,5,0,7,1,3
4,BENE24402,1925-09-01,NaN,2,2,0,39,510,12,12,...,NaN,1,83.0,0,5,5,0,1,1,2


#### Features and target split

#### Train-test split

### Model building

### Model building utilizing graph features

### Conclusion